### Importação de bibliotecas e carregamento da base

Importa todas as dependências necessárias, lê o arquivo `aula_01_exemplo_01.csv`, cria a coluna derivada `tem_filhos` e exibe uma amostra inicial do conjunto de dados.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

df = pd.read_csv('aula_01_exemplo_01.csv')

df['tem_filhos'] = (df['children'] > 0).astype(int)

df.head()


,age,sex,bmi,children,smoker,region,charges,tem_filhos
0,19,female,27.900,0,yes,southwest,16884.92400,0
1,18,male,33.770,1,no,southeast,1725.55230,1
2,28,male,33.000,3,no,southeast,4449.46200,1
3,33,male,22.705,0,no,northwest,21984.47061,0
4,32,male,28.880,0,no,northwest,3866.85520,0


### Definição de matriz de atributos e variável alvo

Separa o dataset em `X`, contendo apenas os atributos preditores, e `Y`, com o alvo `charges`, além de mostrar rapidamente os primeiros registros de cada estrutura.

In [3]:
X = df.drop('charges', axis = 1)
Y = df['charges']

Y.head()
X.head()

,age,sex,bmi,children,smoker,region,tem_filhos
0,19,female,27.900,0,yes,southwest,0
1,18,male,33.770,1,no,southeast,1
2,28,male,33.000,3,no,southeast,1
3,33,male,22.705,0,no,northwest,0
4,32,male,28.880,0,no,northwest,0


### Divisão entre conjuntos de treino e teste

Realiza a partição estratificada do conjunto de dados em bases de treino e teste usando `train_test_split` e informa a quantidade de linhas e colunas resultante de cada partição.

In [4]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X, Y, test_size=0.2, random_state=42)

print('Número de linhas e colunas da base de treino: ', x_treino.shape)
print('Número de linhas e colunas da base de teste: ', x_teste.shape)

Número de linhas e colunas da base de treino:  (1070, 7)
Número de linhas e colunas da base de teste:  (268, 7)


### Seleção de colunas numéricas e categóricas

Define explicitamente quais colunas serão tratadas como numéricas e quais serão consideradas categóricas durante as etapas de pré-processamento.

In [5]:
variaveis_numericas = ['age', 'bmi', 'children', 'tem_filhos']
variaveis_categoricas = ['sex', 'smoker', 'region']

### Configuração dos transformadores base

Instancia os objetos de imputação e transformação, incluindo `StandardScaler`, `SimpleImputer` para dados numéricos e categóricos, além do `OneHotEncoder` para codificar variáveis qualitativas.

In [6]:
escalonador = StandardScaler()

imputador_numerico = SimpleImputer(strategy='median')

categorizador = OneHotEncoder(drop='first', handle_unknown='ignore')

imputador_categorico = SimpleImputer(strategy='most_frequent')

### Pipeline para atributos numéricos

Monta um pipeline que primeiro trata valores ausentes nas colunas numéricas com a mediana e, em seguida, aplica a padronização dos dados.

In [7]:
etapas_numericas = Pipeline (
    [
        ("imputer", imputador_numerico),
        ('scaler', escalonador)
    ]
)

etapas_numericas

,steps,"[('imputer', ...), ('scaler', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


### Pipeline para atributos categóricos

Constrói o pipeline responsável por imputar categorias faltantes com o valor mais frequente e aplicar codificação one-hot nas variáveis categóricas.

In [8]:
etapas_categoricas = Pipeline(
    [
        ('imputer', imputador_categorico),
        ('enconder', categorizador)
    ]
)

etapas_categoricas

,steps,"[('imputer', ...), ('enconder', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,categories,'auto'


### ColumnTransformer e transformação dos dados

Integra os pipelines numérico e categórico em um `ColumnTransformer`, ajusta-o nos dados de treino e gera matrizes transformadas para treino e teste, permitindo verificar o formato final após as etapas de pré-processamento.

In [9]:
preprocessador = ColumnTransformer(
    [
        ('num', etapas_numericas, variaveis_numericas),
        ('cat', etapas_categoricas, variaveis_categoricas)
    ]
)

x_treino_transformado = preprocessador.fit_transform(x_treino)
x_teste_transformado = preprocessador.transform(x_teste)

print(f'Formato do treino transformado: {x_treino_transformado.shape}')
print(f'Formato do teste transformado: {x_teste_transformado.shape}')

if hasattr(x_treino_transformado, 'toarray'):
    primeira_linha = x_treino_transformado[0].toarray().ravel()
else:
    primeira_linha = x_treino_transformado[0]

print('Primeira linha transformada:', primeira_linha)


      age     sex     bmi  children smoker     region  tem_filhos
560    46  female  19.950         2     no  northwest           1
1285   47  female  24.320         0     no  northeast           0
1142   52  female  24.860         0     no  southeast           0
969    39  female  34.320         5     no  southeast           1
486    54  female  21.470         3     no  northwest           1
...   ...     ...     ...       ...    ...        ...         ...
1095   18  female  31.350         4     no  northeast           1
1130   39  female  23.870         5     no  southeast           1
1294   58    male  25.175         0     no  northeast           0
860    37  female  47.600         2    yes  southwest           1
1126   55    male  29.900         0     no  southwest           0

[1070 rows x 7 columns]


array([ 0.47222651, -1.75652513,  0.73433626,  0.86178362,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ])

### Instanciação do modelo de árvore de decisão

Cria o estimador `DecisionTreeRegressor` que será ajustado aos dados pré-processados.

In [10]:
modelo = DecisionTreeRegressor(random_state=42)

### Definição da grade de hiperparâmetros

Especifica os valores candidatos de profundidade máxima e tamanho mínimo de divisão que serão avaliados durante a busca por hiperparâmetros.

In [11]:
param_grid = {
    'max_depth': [2, 3, 5, 7],
    'min_samples_split': [2, 5, 10, 15, 20, 25]
}

### Configuração da busca em grade

Inicializa o `GridSearchCV` com o modelo, a grade de hiperparâmetros, o esquema de validação cruzada e a métrica de avaliação baseada em RMSE negativo.

In [12]:
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

### Treinamento com validação cruzada

Executa o ajuste do `GridSearchCV` sobre os dados transformados de treino e captura o melhor estimador encontrado.

In [13]:
grid_search.fit(x_treino_transformado, y_treino)

melhor_modelo_arvore = grid_search.best_estimator_

### Avaliação da árvore de decisão no treino

Aplica o melhor modelo de árvore obtido na busca em grade aos dados de treino transformados para calcular métricas de desempenho e inspecionar possíveis sinais de overfitting.

In [27]:
y_pred_treino_arvore = melhor_modelo_arvore.predict(x_treino_transformado)

mse_treino_arvore = mean_squared_error(y_treino, y_pred_treino_arvore)
rmse_treino_arvore = np.sqrt(mse_treino_arvore)
mae_treino_arvore = mean_absolute_error(y_treino, y_pred_treino_arvore)
r2_treino_arvore = r2_score(y_treino, y_pred_treino_arvore)
mape_treino_arvore = np.mean(np.abs((y_treino - y_pred_treino_arvore) / y_treino)) * 100

print('Melhores parâmetros encontrados (árvore): ', grid_search.best_params_)
print('Avaliação do modelo de Árvore de Decisão no treino:')
print('R2: ', np.round(r2_treino_arvore, 3))
print('MSE: ', np.round(mse_treino_arvore, 3))
print('RMSE: ', np.round(rmse_treino_arvore, 3))
print('MAE: ', np.round(mae_treino_arvore, 3))
print('MAPE (%): ', np.round(mape_treino_arvore, 2))


Melhores parâmetros encontrados:  {'max_depth': 3, 'min_samples_split': 2}
R2 no treino:  0.854
MSE no treino:  21120357.016
RMSE no treino:  4595.689
MAE no treino:  2785.253
MAPE no treino (%):  34.69


### Instanciação do modelo KNN

Cria o estimador k-vizinhos mais próximos padrão, que servirá como base para a busca de hiperparâmetros.

In [15]:
modelo_knn = KNeighborsRegressor()

### Grade de hiperparâmetros do KNN

Define a quantidade de vizinhos (`n_neighbors`) e a métrica de distância (`p`) que serão avaliadas na validação cruzada.

In [18]:
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'p': [1, 2]
}

### Treinamento do KNN com validação cruzada

Executa o `GridSearchCV` para encontrar a combinação de hiperparâmetros do KNN que minimiza o RMSE médio nas dobras de treino.

In [19]:
grid_search_knn = GridSearchCV(
    estimator=modelo_knn,
    param_grid=param_grid_knn,
    cv=5,
    scoring='neg_root_mean_squared_error'
)

grid_search_knn.fit(x_treino_transformado, y_treino)

melhor_modelo_knn = grid_search_knn.best_estimator_

### Avaliação do KNN no treino

Calcula métricas sobre os dados de treino transformados usando o melhor modelo KNN encontrado.

In [26]:
y_pred_treino_knn = melhor_modelo_knn.predict(x_treino_transformado)

mse_treino_knn = mean_squared_error(y_treino, y_pred_treino_knn)
rmse_treino_knn = np.sqrt(mse_treino_knn)
mae_treino_knn = mean_absolute_error(y_treino, y_pred_treino_knn)
r2_treino_knn = r2_score(y_treino, y_pred_treino_knn)
mape_treino_knn = np.mean(np.abs((y_treino - y_pred_treino_knn) / y_treino)) * 100

print('Melhores parâmetros encontrados (KNN): ', grid_search_knn.best_params_)
print('Avaliação do modelo KNN no treino:')
print('R2: ', np.round(r2_treino_knn, 3))
print('MSE: ', np.round(mse_treino_knn, 3))
print('RMSE: ', np.round(rmse_treino_knn, 3))
print('MAE: ', np.round(mae_treino_knn, 3))
print('MAPE (%): ', np.round(mape_treino_knn, 2))


Melhores parâmetros encontrados:  {'n_neighbors': 3, 'p': 1}
R2 no treino 0.861
MSE no treino 20116314.447
RMSE no treino 4485.121
MAE no treino 2463.82
MAPE no treino (%) 25.87


### Avaliação dos modelos no conjunto de teste

Aplica os modelos de árvore de decisão e KNN aos dados de teste transformados, registrando as métricas de regressão para comparar o desempenho em dados não vistos.

In [ ]:
y_pred_teste_arvore = melhor_modelo_arvore.predict(x_teste_transformado)

mse_teste_arvore = mean_squared_error(y_teste, y_pred_teste_arvore)
rmse_teste_arvore = np.sqrt(mse_teste_arvore)
mae_teste_arvore = mean_absolute_error(y_teste, y_pred_teste_arvore)
r2_teste_arvore = r2_score(y_teste, y_pred_teste_arvore)
mape_teste_arvore = np.mean(np.abs((y_teste - y_pred_teste_arvore) / y_teste)) * 100

print('Avaliação do modelo de Árvore de Decisão no teste:')
print('R2: ', np.round(r2_teste_arvore, 3))
print('MSE: ', np.round(mse_teste_arvore, 3))
print('RMSE: ', np.round(rmse_teste_arvore, 3))
print('MAE: ', np.round(mae_teste_arvore, 3))
print('MAPE (%): ', np.round(mape_teste_arvore, 2))


Avaliação do modelo de Árvore de Decição no teste:  4776.0
R2:  0.853
MSE:  22812669.852
RMSE:  4776.261
MAE:  2865.638
MAPE (%):  37.68


In [30]:
y_pred_teste_knn = melhor_modelo_knn.predict(x_teste_transformado)

mse_teste_knn = mean_squared_error(y_teste, y_pred_teste_knn)
rmse_teste_knn = np.sqrt(mse_teste_knn)
mae_teste_knn = mean_absolute_error(y_teste, y_pred_teste_knn)
r2_teste_knn = r2_score(y_teste, y_pred_teste_knn)
mape_teste_knn = np.mean(np.abs((y_teste - y_pred_teste_knn) / y_teste)) * 100

print('Avaliação do modelo KNN no teste:')
print('R2: ', np.round(r2_teste_knn, 3))
print('MSE: ', np.round(mse_teste_knn, 3))
print('RMSE: ', np.round(rmse_teste_knn, 3))
print('MAE: ', np.round(mae_teste_knn, 3))
print('MAPE (%): ', np.round(mape_teste_knn, 2))


Avaliação do modelo KNN no teste:
R2:  0.708
MSE:  45388801.136
RMSE:  6737.121
MAE:  3996.595
MAPE (%):  42.63


### Conclusões

Os dois modelos compartilham o mesmo pipeline de pré-processamento, mas apresentam comportamentos distintos: a árvore de decisão tende a sobreajustar com facilidade, enquanto o KNN depende fortemente da escolha de vizinhos e da padronização. Compare as métricas obtidas em treino e teste para identificar qual abordagem generaliza melhor para este conjunto de dados.